In [29]:
import pygsti
import numpy as np
from scipy.linalg import expm

In [30]:
# Gell-Mann matrices
gellmann_matrices = [
    np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]),
    np.array([[0, 1, 0], [1, 0, 0], [0, 0, 0]]),
    np.array([[0, -1j, 0], [1j, 0, 0], [0, 0, 0]]),
    np.array([[1, 0, 0], [0, -1, 0], [0, 0, 0]]),
    np.array([[0, 0, 1], [0, 0, 0], [1, 0, 0]]),
    np.array([[0, 0, -1j], [0, 0, 0], [1j, 0, 0]]),
    np.array([[0, 0, 0], [0, 0, 1], [0, 1, 0]]),
    np.array([[0, 0, 0], [0, 0, -1j], [0, 1j, 0]]),
    np.array([[1, 0, 0], [0, 1, 0], [0, 0, -2]])
]

gellmann_8_12 = np.array([[-2, 0, 0], [0, 1, 0], [0, 0, 1]])

In [40]:
# unitary models 
# we ignore axis error 
def modelX01(theta, gamma):
    return expm(-(1j/2)*((np.pi/2 + theta)*gellmann_matrices[1] + gamma*gellmann_matrices[8]))

def modelZ01():
    return expm(-(1j*np.pi/4)*gellmann_matrices[3])

Z12_gen = np.array([[0, 0, 0], [0, 1, 0], [0, 0, -1]])

def modelZ12():
    return expm(-(1j*np.pi/4)*Z12_gen)

def modelX12(theta, gamma):
    return expm(-(1j/2)*((np.pi/2 + theta)*gellmann_matrices[1]  + gamma*gellmann_matrices[8]))



In [51]:
def parse_error_vector(x):
    info = {
        'single_qutrit': {
            'Q1': {
                'X01' : x[0],
                'phase01': x[1],
                'X12' : x[2], 
                'phase12': x[3],
            },
            'Q2': {
                'X01' : x[4],
                'phase01': x[5],
                'X12' : x[6],
                'phase12': x[7],
            }
        },
        'two_qutrit': {
            'phi1': x[8],
            'phi2': x[9],
            'phi3': x[10],
            'phi4': x[11],
            'phi5': x[12],
            'phi6': x[13],
            'phi7': x[14],
            'phi8': x[15]
        }  
    }
    return info

def random_error_vector(single_qutrit_rates, two_qutrit_rates):
    q1_vec = np.random.multivariate_normal(np.zeros(4), np.eye(4)*single_qutrit_rates)
    q2_vec = np.random.multivariate_normal(np.zeros(4), np.eye(4)*single_qutrit_rates)
    two_qubit_vec = np.random.multivariate_normal(np.zeros(8), np.eye(8)*two_qutrit_rates)
    return np.concatenate((q1_vec, q2_vec, two_qubit_vec))


In [56]:
from pygsti.tools import unitary_to_std_process_mx
from pygsti.modelmembers.operations import EmbeddedOp, FullUnitaryOp

In [57]:
EmbeddedOp?

Init signature:
EmbeddedOp(
    state_space,
    target_labels,
    operation_to_embed,
    allocated_to_parent=None,
)
Docstring:     
An operation containing a single lower (or equal) dimensional operation within it.

An EmbeddedOp acts as the identity on all of its domain except the
subspace of its contained operation, where it acts as the contained operation does.

Parameters
----------
state_space : StateSpace
    Specifies the density matrix space upon which this operation acts.

target_labels : list of strs
    The labels contained in `state_space` which demarcate the
    portions of the state space acted on by `operation_to_embed` (the
    "contained" operation).

operation_to_embed : LinearOperator
    The operation object that is to be contained within this operation, and
    that specifies the only non-trivial action of the EmbeddedOp.
Init docstring: Initialize a new LinearOperator 
File:           /opt/anaconda3/envs/qubic/lib/python3.12/site-packages/pygsti/modelmembers/o

In [58]:
FullUnitaryOp?

Init signature: FullUnitaryOp(m, basis='pp', evotype='default', state_space=None)
Docstring:     
An operation matrix that is fully parameterized.

That is, each element of the operation matrix is an independent parameter.

Parameters
----------
m : array_like or LinearOperator
    a square 2D array-like or LinearOperator object representing the operation action.
    The shape of m sets the dimension of the operation.

basis : Basis or {'pp','gm','std'}, optional
    The basis used to construct the Hilbert-Schmidt space representation
    of this state as a super-operator.

evotype : Evotype or str, optional
    The evolution type.  The special value `"default"` is equivalent
    to specifying the value of `pygsti.evotypes.Evotype.default_evotype`.

state_space : StateSpace, optional
    The state space for this operation.  If `None` a default state space
    with the appropriate number of qubits is used.
Init docstring: Initialize a new LinearOperator 
File:           /opt/anaconda3/e

In [65]:
from pygsti.baseobjs import ExplicitStateSpace
from pygsti.models import ExplicitOpModel

def make_model(error_vector, single_qutrit_depol, two_qutrit_depol):

    joint_state_space = ExplicitStateSpace([('Q1','Q2')], [(3,3)])
    ss_q1 = ExplicitStateSpace('Q1', 3)
    ss_q2 = ExplicitStateSpace('Q2', 3)

    model = ExplicitOpModel(joint_state_space)

    errors = parse_error_vector(error_vector)
    x01_Q1 = errors['single_qutrit']['Q1']['X01']
    x12_Q1 = errors['single_qutrit']['Q1']['X12']
    x01_Q2 = errors['single_qutrit']['Q2']['X01']
    x12_Q2 = errors['single_qutrit']['Q2']['X12']
    
    phase01_Q1 = errors['single_qutrit']['Q1']['phase01']
    phase12_Q1 = errors['single_qutrit']['Q1']['phase12']
    phase01_Q2 = errors['single_qutrit']['Q2']['phase01']
    phase12_Q2 = errors['single_qutrit']['Q2']['phase12']

    phi1 = errors['two_qutrit']['phi1']
    phi2 = errors['two_qutrit']['phi2']
    phi3 = errors['two_qutrit']['phi3']
    phi4 = errors['two_qutrit']['phi4']
    phi5 = errors['two_qutrit']['phi5']
    phi6 = errors['two_qutrit']['phi6']
    phi7 = errors['two_qutrit']['phi7']
    phi8 = errors['two_qutrit']['phi8']


    # Define single qutrit gates
    X01_Q1_unitary = FullUnitaryOp(unitary_to_std_process_mx(modelX01(x01_Q1, phase01_Q1)), basis='std', state_space=ss_q1)
    Z01_Q1_unitary = FullUnitaryOp(unitary_to_std_process_mx(modelZ01()), basis='std', state_space=ss_q1)
    X12_Q1_unitary = FullUnitaryOp(unitary_to_std_process_mx(modelX12(x12_Q1, phase12_Q1)), basis='std', state_space=ss_q1)
    Z12_Q1_unitary = FullUnitaryOp(unitary_to_std_process_mx(modelZ12()), basis='std', state_space=ss_q1)

    X01_Q2_unitary = FullUnitaryOp(unitary_to_std_process_mx(modelX01(x01_Q2, phase01_Q2)), basis='std', state_space=ss_q2)
    Z01_Q2_unitary = FullUnitaryOp(unitary_to_std_process_mx(modelZ01()), basis='std', state_space=ss_q2)
    X12_Q2_unitary = FullUnitaryOp(unitary_to_std_process_mx(modelX12(x12_Q2, phase12_Q2)), basis='std', state_space=ss_q2)
    Z12_Q2_unitary = FullUnitaryOp(unitary_to_std_process_mx(modelZ12()), basis='std', state_space=ss_q2)

    model.operations['X01(Q1)'] = EmbeddedOp(joint_state_space, ('Q1',), X01_Q1_unitary).depolarize(single_qutrit_depol)
    model.operations['Z01(Q1)'] = EmbeddedOp(joint_state_space, ('Q1',), Z01_Q1_unitary).depolarize(single_qutrit_depol)
    model.operations['X12(Q1)'] = EmbeddedOp(joint_state_space, ('Q1',), X12_Q1_unitary).depolarize(single_qutrit_depol)
    model.operations['Z12(Q1)'] = EmbeddedOp(joint_state_space, ('Q1',), Z12_Q1_unitary).depolarize(single_qutrit_depol)

    model.operations['X01(Q2)'] = EmbeddedOp(joint_state_space, ('Q2',), X01_Q2_unitary).depolarize(single_qutrit_depol)
    model.operations['Z01(Q2)'] = EmbeddedOp(joint_state_space, ('Q2',), Z01_Q2_unitary).depolarize(single_qutrit_depol)
    model.operations['X12(Q2)'] = EmbeddedOp(joint_state_space, ('Q2',), X12_Q2_unitary).depolarize(single_qutrit_depol)
    model.operations['Z12(Q2)'] = EmbeddedOp(joint_state_space, ('Q2',), Z12_Q2_unitary).depolarize(single_qutrit_depol)


    return model

In [66]:
x = random_error_vector(0.1, 0.1)
model = make_model(x, 0.1, 0.1)

AssertionError: dest-basis dimension mismatch: 9 != 81